In [1]:
import numpy as np
from scipy.stats import kurtosis, mode, skew
from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression   
import neurolab as nl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [92]:
# Para leer el tipo de archivo .data se utiliza la siguiente función
with open('synthetic_control.data') as input_file:
    lines = input_file.readlines()
    newLines = []
    for line in lines:
        newLine = line.strip().split()
        newLines.append(newLine )
dato = newLines
# devuelve una lista de tipo string
# lo transformamos por numeros
datos = np.array(dato).astype(float)

In [3]:
# Definimos las 7 caracteristicas para los datos
def caracteristicas(X):
    M = []
    for i in range(len(X)):
        # Guardamos las 7 caracteristicas para cada observación
        M.append([np.mean(X[i,:]),np.median(X[i,:]),kurtosis(X[i,:]),np.std(X[i,:]),np.var(X[i,:]),skew(X[i,:]),mode(X[i,:])[0][0]])
    return M

In [7]:
# Preprocesamos los datos
def preprocesamiento(X_train,X_test):
    #Remoción para x_train
    def Remocion(X_train):
        med = X_train.mean(axis=0)
        des = X_train.std(axis=0)
        X_train = X_train - med
        X_train = X_train/des
        return X_train,med, des

    #Remocion para x_test
    def Remocion_test(X_test,med,des):
        X_test = X_test - med
        X_test = X_test/des
        return X_test
    #nomralizacion de x_train
    x_norm_train,t_train_mean,t_train_desv = Remocion(X_train)
    #Normalizacion para x_test
    x_norm_test = Remocion_test(X_test,t_train_mean,t_train_desv)
    
    return x_norm_test,x_norm_train

In [88]:
def red_neuronal(X,y):
    # Definimos los valores minimos y maximos para cada dimension
    # Dos neuronas para la entrada y dos para la salida
    dim1_min, dim1_max = X[:,0].min(), X[:,0].max() 
    dim2_min, dim2_max = X[:,1].min(), X[:,1].max() 
    dim3_min, dim3_max = X[:,2].min(), X[:,2].max() 
    dim4_min, dim4_max = X[:,3].min(), X[:,3].max() 
    dim5_min, dim5_max = X[:,4].min(), X[:,4].max() 
    dim6_min, dim6_max = X[:,5].min(), X[:,5].max() 
    dim7_min, dim7_max = X[:,6].min(), X[:,6].max() 
    # definimos las entradas
    dim1 = [dim1_min, dim1_max]
    dim2 = [dim2_min, dim2_max]
    dim3 = [dim3_min, dim3_max]
    dim4 = [dim4_min, dim4_max]
    dim5 = [dim5_min, dim5_max]
    dim6 = [dim6_min, dim6_max]
    dim7 = [dim7_min, dim7_max]
    
    ANN = nl.net.newff([dim1,dim2,dim3,dim4,dim5,dim6,dim7],[2,1,3])
    ANN.trainf = nl.train.train_gd 
    progreso_erro = ANN.train(X_train, y_train, epochs = 1000, show = 200, goal = 0.01)
    

In [95]:
def red_neuronal_3(X,y):
    # Definimos los valores minimos y maximos para cada dimension
    # Dos neuronas para la entrada y dos para la salida
    dim1_min, dim1_max = X[:,0].min(), X[:,0].max() 
    dim2_min, dim2_max = X[:,1].min(), X[:,1].max() 
    dim3_min, dim3_max = X[:,2].min(), X[:,2].max() 
    
    # definimos las entradas
    dim1 = [dim1_min, dim1_max]
    dim2 = [dim2_min, dim2_max]
    dim3 = [dim3_min, dim3_max]
        
    ANN = nl.net.newff([dim1,dim2,dim3],[2,1,3])
    ANN.trainf = nl.train.train_gd 
    progreso_erro = ANN.train(X_train, y_train, epochs = 1000, show = 200, goal = 0.01)
    

In [106]:
# Selector de caracteristicas solo 4
def selectores(X_test,X_train,y_train):
    selector = SelectKBest(f_regression, k=4)
    print(np.shape(X_train),np.shape(y_train))
    selector.fit(X_train,y_train)
    X_test = selector.transform(X_test)
    X_train = selector.transform(X_train)
    return(X_test,X_train)

In [90]:
# Definimos Y
y = []
y = [[0,0,0]]*100 + [[0,0,1]]*100 + [[0,1,0]]*100 + [[0,1,1]]*100 + [[1,0,0]]*100 + [[1,0,1]]*100 
y = np.vstack(y)

In [93]:
# Transformamos las caracteristicas en 
X = np.array(caracteristicas(datos))

In [108]:
# Dividimos los datos
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.3)
# Preprocesamos
X_test,X_train = preprocesamiento(X_train,X_test)
# Selector Caracteristicas 3
y_train_p = y_train[:,0]
X_test,X_train = selectores(X_test,X_train,y_train_p)
# Aplicamos la red neuronal con capas cultas.

red_neuronal_3(X_train,y_train)

(420, 7) (420,)


AssertionError: 